# 04 — Regression (Supervised) for PM2.5 (Tabular Time-Lag Features)
Mục tiêu:
- Biến bài toán chuỗi thời gian thành bài toán **hồi quy có giám sát**: dự đoán PM2.5(t+h) từ đặc trưng tại thời điểm t.
- Thấy rõ: **leakage** và vì sao phải split theo thời gian.
- So sánh tư duy hồi quy (feature-based) vs ARIMA (time-series-based).


In [ ]:
from pathlib import Path

# ===== PARAMETERS =====
USE_UCIMLREPO = False

# Path to the raw ZIP (relative to project root)
RAW_ZIP_PATH = 'data/raw/PRSA2017_Data_20130301-20170228.zip'

def resolve_project_root(raw_zip_rel: str) -> Path:
    """Resolve project root robustly for both Jupyter and Papermill runs.

    Papermill may execute notebooks with different working directories depending on how you run the pipeline.
    This helper tries:
      1) current working directory
      2) parent directory
      3) up to 3 levels up (useful when running from notebooks/ or notebooks/runs)
    """
    cwd = Path.cwd().resolve()

    # Common candidates
    candidates = [cwd, cwd.parent]

    # Climb up a few levels just in case
    root = cwd
    for _ in range(3):
        candidates.append(root)
        root = root.parent

    for r in candidates:
        if (r / raw_zip_rel).exists():
            return r

    # Fallback: keep cwd; downstream will raise a clear error message if missing
    return cwd

PROJECT_ROOT = resolve_project_root(RAW_ZIP_PATH)
RAW_ZIP_ABS = str((PROJECT_ROOT / RAW_ZIP_PATH).resolve())

LAG_HOURS = [1, 3, 24]
HORIZON = 1              # dự đoán trước bao nhiêu giờ
TARGET_COL = 'PM2.5'

OUTPUT_REG_DATASET_PATH = 'data/processed/dataset_for_regression.parquet'
CUTOFF = '2017-01-01'

MODEL_OUT = 'regressor.joblib'
METRICS_OUT = 'regression_metrics.json'
PRED_SAMPLE_OUT = 'regression_predictions_sample.csv'


In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.classification_library import Paths
from src.regression_library import (
    run_prepare_regression_dataset,
    run_train_regression,
)

paths = Paths(project_root=PROJECT_ROOT)
print('PROJECT_ROOT =', PROJECT_ROOT)
print('RAW_ZIP_ABS =', RAW_ZIP_ABS)


## 1) Tạo dataset hồi quy (lag features + time features + y = PM2.5(t+h))
Trong lab, phần này giúp sinh viên hiểu cách tạo supervised dataset từ time series.

In [ ]:
out_path = run_prepare_regression_dataset(
    paths=paths,
    use_ucimlrepo=USE_UCIMLREPO,
    raw_zip_path=RAW_ZIP_ABS,
    lag_hours=LAG_HOURS,
    horizon=HORIZON,
    target_col=TARGET_COL,
)
print('Saved:', out_path)


## 2) Quick EDA cho dataset hồi quy
Gợi ý câu hỏi ra quyết định:
- Tỉ lệ missing ở các feature lag? (thường thiếu ở đầu chuỗi)
- PM2.5 có phân phối lệch (skew) không? -> cân nhắc log/clip (tuỳ chọn)
- Có khác biệt theo *giờ trong ngày* / *ngày trong tuần* không? (seasonality)


In [ ]:
ds_path = (PROJECT_ROOT / OUTPUT_REG_DATASET_PATH).resolve()
df = pd.read_parquet(ds_path)
print(df.shape)
display(df.head())

missing = df.isna().mean().sort_values(ascending=False).head(15)
display(missing)

plt.figure()
pd.Series(df[TARGET_COL]).dropna().plot(kind='hist', bins=60)
plt.title(f'Distribution of {TARGET_COL} (raw)')
plt.show()


## 3) Train/Test theo thời gian + train regressor
Lưu ý: mô hình hồi quy ở đây là **feature-based** (dùng lag + thời tiết).
Phần dự báo chuỗi thời gian *thuần* sẽ làm bằng ARIMA ở notebook kế tiếp.

In [ ]:
out = run_train_regression(
    paths=paths,
    cutoff=CUTOFF,
    model_out=MODEL_OUT,
    metrics_out=METRICS_OUT,
    preds_out=PRED_SAMPLE_OUT,
)

print('Metrics:')
print(json.dumps(out['metrics'], ensure_ascii=False, indent=2))
pred_df = out['pred_df']
display(pred_df.head())

# Plot a small window for storytelling
sample = pred_df.dropna().iloc[:500].copy()
plt.figure(figsize=(10,4))
plt.plot(sample['datetime'], sample['y_true'], label='Actual')
plt.plot(sample['datetime'], sample['y_pred'], label='Predicted')
plt.title('Regression: Actual vs Predicted (sample)')
plt.legend()
plt.tight_layout()
plt.show()
